# Sprint 機械学習フロー

## 1.このSprintについて
### Sprintの目的
- 機械学習の実践的な流れを知る
- 高い汎化性能を持つモデルを完成させる

### どのように学ぶか
丁寧な検証が行える状態にした上で、他者の解法を参考に汎化性能の高いモデル作りを進めます。

## 2.機械学習フロー
Kaggleの Home Credit Default Risk コンペティションを題材に、機械学習の実践的な流れを学びます。<br>
特に適切な 検証 を行い、高い 汎化性能 のあるモデルを完成させることを目指します。<br>

### 【問題1】クロスバリデーション
事前学習期間では検証データをはじめに分割しておき、それに対して指標値を計算することで検証を行っていました。（ホールドアウト法）<br>
しかし、分割の仕方により精度は変化します。実践的には クロスバリデーション（交差検証） を行います。<br>
分割を複数回行い、それぞれに対して学習と検証を行う方法です。複数回の分割のためにscikit-learnにはKFoldクラスが用意されています。<br>
事前学習期間の課題で作成したベースラインモデルに対してKFoldクラスによるクロスバリデーションを行うコードを作成し実行してください。<br>

In [1]:
import numpy as np
import pandas as pd

In [2]:
train_df = pd.read_csv("application_train.csv")
test_df = pd.read_csv("application_test.csv")
df_copy = test_df.copy(deep=False)

In [3]:
train_df.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
# 学習データから目的変数を分ける
X_train_df = train_df.iloc[:, 2:]
y_train_df = train_df.iloc[:, 1:2]
# 学習データの説明変数とテストデータを結合
X_all_df = pd.concat([X_train_df, test_df]) 

/Users/arisa/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """


In [5]:
X_train_df.head()

,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,Family,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,Unaccompanied,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,297000.0,Unaccompanied,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,513000.0,Unaccompanied,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
y_train_df.head()

,TARGET
0,1
1,0
2,0
3,0
4,0


In [7]:
# 学習に使う特徴量を絞る
index_array = X_all_df.columns
index_list = index_array.tolist()
index_list.remove("DAYS_BIRTH")
index_list.remove("EXT_SOURCE_1")
index_list.remove("EXT_SOURCE_2")
index_list.remove("EXT_SOURCE_3")
X_all_df.drop(columns=index_list, inplace=True)

In [8]:
# 欠損値を確認する
X_all_df.isnull().sum()

DAYS_BIRTH           0
EXT_SOURCE_1    193910
EXT_SOURCE_2       668
EXT_SOURCE_3     69633
dtype: int64

In [9]:
# 欠損値を0で埋める
X_all_df.fillna(0, inplace=True)

In [10]:
X_all_df.isnull().sum()

DAYS_BIRTH      0
EXT_SOURCE_1    0
EXT_SOURCE_2    0
EXT_SOURCE_3    0
dtype: int64

In [11]:
# 結合データをtrainデータとtestデータに分ける
X_train_df = X_all_df.iloc[:train_df.shape[0], :]
X_test_df = X_all_df.iloc[train_df.shape[0]:, :]

In [12]:
# df→ndarray
X_train = X_train_df.values
X_test = X_test_df.values
# y.shape(sample,)に型変換
y_train = y_train_df.values
y_train = y_train.reshape(y_train.shape[0])

In [13]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)

(307511, 4)
(48744, 4)
(307511,)


【ROC曲線について】
ROC曲線(Receiver Operatorating Characteristic curve、受信者動作特性曲線)は、もともとレーダーシステムの通信工学理論として開発されたものであり、レーダー信号のノイズの中から敵機の存在を検出するための方法として開発された方法です。<br>
ROC解析を行う前提条件としては、説明変数が連続変数であり、アウトカムである目的変数が二分変数であるという事です<br>
参考<br>
http://www.med.osaka-u.ac.jp/pub/kid/clinicaljournalclub6.html

In [14]:
# KFoldクラスでtrainデータをクロスバリデーションで分割
# 標準化
# ロジスティック回帰で学習
# ROC曲線下面積(AUC)で評価:スコアが高い方が良いモデル
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

kf = KFold(n_splits=4, shuffle=True, random_state=71)
scores = [] 
for tr_idx, va_idx in kf.split(X_train):
    tr_x, va_x = X_train[tr_idx], X_train[va_idx]
    tr_y, va_y = y_train[tr_idx], y_train[va_idx]
    # 標準化
    scaler_x = StandardScaler()
    # Xのみ標準化
    scaler_x.fit(tr_x)
    tr_x_std = scaler_x.transform(tr_x)
    va_x_std = scaler_x.transform(va_x)
    
    # 学習の実行、バリデーションデータの予測値の出力、スコア計算
    # ロジスティック回帰
    model = LogisticRegression()
    model.fit(tr_x_std, tr_y)
    # predict_probaで1の確率を配列で渡す
    va_pred = model.predict_proba(va_x_std)[:, 1]
    # roc aucで評価
    score = roc_auc_score(va_y, va_pred)
    scores.append(score)
    
print(np.mean(scores))

0.7017859086738075


### 【問題2】グリッドサーチ
scikit-learnのGridSearchCVを使い、グリッドサーチを行うコードを作成してください。そして、ベースラインモデルに対して何らかしらのパラメータチューニングを行なってください。どのパラメータをチューニングするかは、使用した手法の公式ドキュメントを参考にしてください。

【ロジスティック回帰の計算方法】<br>
$y = S(u)$<br>
ニューロンの出力yは、uの値をシグモイド関数に欠けた結果<br>
$u = \sum_{w_i,x_i} + B$<br>
u とは、それぞれの入力データ$x_i$と重み$w_i$の積の合計に、バイアスBを加えた値<br>
$S(u) = \frac{1}{1 + exp(-u)}$<br>
本参照

ロジスティック回帰について
最適化問題として、バイナリクラス 　ペナルティ付きロジスティック回帰は，次のコスト関数を最小化する．
$$
\min_{w, c} \frac{1}{2}w^T w + C \sum_{i=1}^n \log(\exp(- y_i (X_i^T w + c)) + 1) .
$$
https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression

【正則化】
正則化はモデルのパラメータの学習に使われ、特に過学習を防ぎ、汎化能力を高めるために使われる。<br>
機械学習において最も一般的なのは L1 正則化 (p=1) と L2 正則化 (p=2) である。損失関数 ${\displaystyle E({\boldsymbol {w}})}$の代わりに、<br>
$E({\boldsymbol {w}})+\lambda {\frac {1}{p}}\sum _{i}|w_{i}|^{p}$
を使用する。<br>
L2の場合ならp=2なので<br>
$E({\boldsymbol {w}})+\lambda {\frac {1}{2}}\sum _{i}|w_{i}|^{2}$<br>
$\frac{1}{2}$かけるのは微分しやすいように<br>
今回のCは逆数なので$C=1÷\lambda = \frac {1}{\lambda}$であり、正の値。<br>
一般的に0.001、0.01など$10^{-10*x}$の数が入る
参考：https://ja.wikipedia.org/wiki/%E6%AD%A3%E5%89%87%E5%8C%96
https://ai-trend.jp/basic-study/neural-network/regularization/

In [15]:
# trainデータを訓練データ（train）75%、検証データ(var)25%で分割
from sklearn.model_selection import train_test_split
tr_x, va_x, tr_y, va_y = train_test_split(X_train_df, y_train_df, test_size=0.25, random_state=0)

In [16]:
# y.shape(sample,)に型変換
tr_y = tr_y.values.reshape(tr_y.shape[0])
va_y = va_y.values.reshape(va_y.shape[0])

In [17]:
va_y.shape

(76878,)

In [18]:
# Xのみ標準化
from sklearn.preprocessing import StandardScaler
scaler_x = StandardScaler()
scaler_x.fit(tr_x)
tr_x_std = scaler_x.transform(tr_x)
va_x_std = scaler_x.transform(va_x)

In [19]:
# train_test_splitで分割したtrainデータでグリッドサーチ
from sklearn.model_selection import GridSearchCV
parameters = {"penalty":["l1","l2"],"C":[0.001, 0.01,0.1]}
logreg= LogisticRegression()
clf = GridSearchCV(logreg, parameters)
clf.fit(tr_x_std, tr_y)
# predict_probaで1の確率を配列で渡す
va_pred = clf.predict_proba(tr_x_std)[:, 1]
# roc aucで評価
score = roc_auc_score(tr_y, va_pred)
print(score)

0.7013727730104501


【考察】
同じ条件で、グリッドサーチをしなかった場合のRUCスコアは0.701192403718053だったので0.00004ほどしか上がらなかった

In [20]:
# 分割前のXのみ標準化
scaler = StandardScaler()
scaler.fit(X_train)
X_train_std = scaler.transform(X_train)

In [21]:
# グリッドサーチ+クロスバリデーション4のときの結果も確認
parameters = {"penalty":["l1","l2"],"C":[0.001, 0.01,0.1]}
logreg= LogisticRegression()
clf = GridSearchCV(logreg, parameters, cv=4)
clf.fit(X_train_std, y_train)
# predict_probaで1の確率を配列で渡す
va_pred = clf.predict_proba(X_train_std)[:, 1]
# roc aucで評価
score = roc_auc_score(y_train, va_pred)
print(score)

0.7015516811934406


【考察】<br>
クロスバリデーションも行うと、グリッドサーチともにしなかった時に比べ、0.0002ほど上がった

In [22]:
pd.DataFrame.from_dict(clf.cv_results_)

/Users/arisa/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/arisa/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/arisa/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnin

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_penalty,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,mean_train_score,std_train_score
0,0.201597,0.002668,0.005401,0.000567,0.001,l1,"{'C': 0.001, 'penalty': 'l1'}",0.919263,0.919275,0.919274,0.919274,0.919271,0.000005,1,0.919274,0.91927,0.91927,0.91927,0.919271,0.000002
1,0.185024,0.001575,0.005259,0.000362,0.001,l2,"{'C': 0.001, 'penalty': 'l2'}",0.919263,0.919275,0.919274,0.919274,0.919271,0.000005,1,0.919274,0.91927,0.91927,0.91927,0.919271,0.000002
2,0.204581,0.005383,0.005289,0.000263,0.01,l1,"{'C': 0.01, 'penalty': 'l1'}",0.919263,0.919275,0.919274,0.919274,0.919271,0.000005,1,0.919274,0.91927,0.91927,0.91927,0.919271,0.000002
3,0.181204,0.005183,0.005258,0.000180,0.01,l2,"{'C': 0.01, 'penalty': 'l2'}",0.919263,0.919275,0.919274,0.919274,0.919271,0.000005,1,0.919274,0.91927,0.91927,0.91927,0.919271,0.000002
4,0.211123,0.005383,0.005250,0.000285,0.1,l1,"{'C': 0.1, 'penalty': 'l1'}",0.919263,0.919275,0.919274,0.919274,0.919271,0.000005,1,0.919274,0.91927,0.91927,0.91927,0.919271,0.000002
5,0.184339,0.000761,0.004975,0.000183,0.1,l2,"{'C': 0.1, 'penalty': 'l2'}",0.919263,0.919275,0.919274,0.919274,0.919271,0.000005,1,0.919274,0.91927,0.91927,0.91927,0.919271,0.000002


In [23]:
clf.best_params_

{'C': 0.001, 'penalty': 'l1'}

## 【問題3】Kaggle Notebooksからの調査
KaggleのNotebooksから様々なアイデアを見つけ出して、列挙してください。

- LightGBMを使用している人が多い（Microsoftが提供している決定木の勾配ブースティング）　google colabではそのまま使える
- StratifiedKFoldを使用している人もいた（目的変数「TARGET」の値が1の割合が1割程度と少なかったので、比べてみるのも良いかも）
- 欠損値を平均した値で埋めている人がいた
- ランダムフォレストを使用している人もいた

 ## 【問題4】高い汎化性能のモデル作成
問題3で見つけたアイデアと、独自のアイデアを組み合わせ高い汎化性能のモデル作りを進めてください。<br>
その過程として、何を行うことで、クロスバリデーションの結果がどの程度変化したかを表にまとめてください。

- 決定木のバギングがランダムフォレストなので、モデルをランダムフォレストにしてみる
- StratifiedKFoldを使用してみる
- 欠損値を平均した値で埋める
- グリッドサーチのCの値に0.0001を追加してみる
- 比較のベースモデルはクロスバリデーション4、ロジスティック回帰C: 0.001, penalty: l1、欠損値は0で埋める

In [24]:
df = pd.DataFrame({"base":score}, index=["ROCAUC"])

In [25]:
# クロスバリデーション4+ランダムフォレスト
from sklearn.ensemble import RandomForestClassifier
kf = KFold(n_splits=4, shuffle=True, random_state=71)
scores = [] 
for tr_idx, va_idx in kf.split(X_train):
    tr_x, va_x = X_train[tr_idx], X_train[va_idx]
    tr_y, va_y = y_train[tr_idx], y_train[va_idx]
    # 標準化
    scaler_x = StandardScaler()
    # Xのみ標準化
    scaler_x.fit(tr_x)
    tr_x_std = scaler_x.transform(tr_x)
    va_x_std = scaler_x.transform(va_x)
    
    # 学習の実行、バリデーションデータの予測値の出力、スコア計算
    # ランダムフォレスト
    model = RandomForestClassifier(n_estimators=50, max_depth=5, min_samples_split=2, min_samples_leaf=1)
    model.fit(tr_x_std, tr_y)
    # predict_probaで1の確率を配列で渡す
    va_pred = model.predict_proba(va_x_std)[:, 1]
    # roc aucで評価
    score = roc_auc_score(va_y, va_pred)
    scores.append(score)
    
print(np.mean(scores))

/Users/arisa/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


0.7193368574414214


In [26]:
df = df.assign(ランダムフォレスト=[np.mean(scores)])
df

,base,ランダムフォレスト
ROCAUC,0.701552,0.719337


In [27]:
# StratifiedKFoldクラスでtrainデータを層化クロスバリデーションで分割
# 標準化
# ロジスティック回帰で学習
# ROC曲線下面積(AUC)で評価:スコアが高い方が良いモデル
from sklearn.model_selection import StratifiedKFold

skf = KFold(n_splits=4, shuffle=True, random_state=71)
scores = [] 
for tr_idx, va_idx in skf.split(X_train):
    tr_x, va_x = X_train[tr_idx], X_train[va_idx]
    tr_y, va_y = y_train[tr_idx], y_train[va_idx]
    # 標準化
    scaler_x = StandardScaler()
    # Xのみ標準化
    scaler_x.fit(tr_x)
    tr_x_std = scaler_x.transform(tr_x)
    va_x_std = scaler_x.transform(va_x)
    
    # 学習の実行、バリデーションデータの予測値の出力、スコア計算
    # ロジスティック回帰 
    model = LogisticRegression(C=0.001, penalty="l1")
    model.fit(tr_x_std, tr_y)
    # predict_probaで1の確率を配列で渡す
    va_pred = model.predict_proba(va_x_std)[:, 1]
    # roc aucで評価
    score = roc_auc_score(va_y, va_pred)
    scores.append(score)
    
print(np.mean(scores))

0.7011298583091462


In [28]:
df = df.assign(層化CV=[np.mean(scores)])
df

,base,ランダムフォレスト,層化CV
ROCAUC,0.701552,0.719337,0.70113


In [29]:
# グリッドサーチ+CV:Cの値に0.0001を追加してみる
parameters = {"penalty":["l1","l2"],"C":[0.0001, 0.001]}
logreg= LogisticRegression()
clf = GridSearchCV(logreg, parameters, cv=4)
clf.fit(X_train_std, y_train)
# predict_probaで1の確率を配列で渡す
va_pred = clf.predict_proba(X_train_std)[:, 1]
# roc aucで評価
score = roc_auc_score(y_train, va_pred)
print(score)

0.6554299749021755


In [30]:
clf.best_params_

{'C': 0.0001, 'penalty': 'l1'}

In [35]:
df = df.assign(paramC=[score])
df

,base,ランダムフォレスト,層化CV,paramC
ROCAUC,0.701552,0.719337,0.70113,0.65543


In [36]:
# 読み込み直して、欠損値を平均で埋める
train_df = pd.read_csv("application_train.csv")
test_df = pd.read_csv("application_test.csv")
df_copy = test_df.copy(deep=False)

In [37]:
# 学習データから目的変数を分ける
X_train_df = train_df.iloc[:, 2:]
y_train_df = train_df.iloc[:, 1:2]
# 学習データの説明変数とテストデータを結合
X_all_df = pd.concat([X_train_df, test_df]) 

/Users/arisa/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """


In [38]:
# 学習に使う特徴量を絞る
index_array = X_all_df.columns
index_list = index_array.tolist()
index_list.remove("DAYS_BIRTH")
index_list.remove("EXT_SOURCE_1")
index_list.remove("EXT_SOURCE_2")
index_list.remove("EXT_SOURCE_3")
X_all_df.drop(columns=index_list, inplace=True)

In [39]:
# 欠損値を確認する
X_all_df.isnull().sum()

DAYS_BIRTH           0
EXT_SOURCE_1    193910
EXT_SOURCE_2       668
EXT_SOURCE_3     69633
dtype: int64

In [43]:
X_all_df.mean(axis=0)

DAYS_BIRTH     -16041.248841
EXT_SOURCE_1        0.501965
EXT_SOURCE_2        0.514890
EXT_SOURCE_3        0.509350
dtype: float64

In [46]:
X_all_df = X_all_df.fillna({"EXT_SOURCE_1":0.501965, "EXT_SOURCE_2":0.514890, "EXT_SOURCE_3":0.509350})

In [49]:
# 結合データをtrainデータとtestデータに分ける
X_train_df = X_all_df.iloc[:train_df.shape[0], :]
X_test_df = X_all_df.iloc[train_df.shape[0]:, :]

In [50]:
# df→ndarray
X_train = X_train_df.values
X_test = X_test_df.values
# y.shape(sample,)に型変換
y_train = y_train_df.values
y_train = y_train.reshape(y_train.shape[0])

In [61]:
# 分割前のXのみ標準化
scaler = StandardScaler()
scaler.fit(X_train)
X_train_std = scaler.transform(X_train)
X_test_std = scaler.transform(X_test)

In [52]:
# グリッドサーチ+クロスバリデーション4のときの結果も確認
parameters = {"penalty":["l1","l2"],"C":[0.001, 0.01,0.1]}
logreg= LogisticRegression()
clf = GridSearchCV(logreg, parameters, cv=4)
clf.fit(X_train_std, y_train)
# predict_probaで1の確率を配列で渡す
va_pred = clf.predict_proba(X_train_std)[:, 1]
# roc aucで評価
score = roc_auc_score(y_train, va_pred)
print(score)

0.7200057708530866


In [53]:
clf.best_params_

{'C': 0.001, 'penalty': 'l2'}

In [54]:
df = df.assign(平均値で置換=[score])
df

,base,ランダムフォレスト,層化CV,paramC,平均値で置換
ROCAUC,0.701552,0.719337,0.70113,0.65543,0.720006


平均値で欠損値を置換+ランダムフォレストでより良いモデルになりそうなので検証する

In [55]:
# クロスバリデーション4+ランダムフォレスト
kf = KFold(n_splits=4, shuffle=True, random_state=71)
scores = [] 
for tr_idx, va_idx in kf.split(X_train):
    tr_x, va_x = X_train[tr_idx], X_train[va_idx]
    tr_y, va_y = y_train[tr_idx], y_train[va_idx]
    # 標準化
    scaler_x = StandardScaler()
    # Xのみ標準化
    scaler_x.fit(tr_x)
    tr_x_std = scaler_x.transform(tr_x)
    va_x_std = scaler_x.transform(va_x)
    
    # 学習の実行、バリデーションデータの予測値の出力、スコア計算
    # ランダムフォレスト
    model = RandomForestClassifier(n_estimators=50, max_depth=5, min_samples_split=2, min_samples_leaf=1)
    model.fit(tr_x_std, tr_y)
    # predict_probaで1の確率を配列で渡す
    va_pred = model.predict_proba(va_x_std)[:, 1]
    # roc aucで評価
    score = roc_auc_score(va_y, va_pred)
    scores.append(score)
    
print(np.mean(scores))

0.7210132041110708


In [56]:
df = df.assign(平均値で置換とランダムF=[np.mean(scores)])
df

,base,ランダムフォレスト,層化CV,paramC,平均値で置換,平均値で置換とランダムF
ROCAUC,0.701552,0.719337,0.70113,0.65543,0.720006,0.721013


ランダムフォレストのパラメータを微調整

In [57]:
# クロスバリデーション4+ランダムフォレストn_estimators=100
kf = KFold(n_splits=4, shuffle=True, random_state=71)
scores = [] 
for tr_idx, va_idx in kf.split(X_train):
    tr_x, va_x = X_train[tr_idx], X_train[va_idx]
    tr_y, va_y = y_train[tr_idx], y_train[va_idx]
    # 標準化
    scaler_x = StandardScaler()
    # Xのみ標準化
    scaler_x.fit(tr_x)
    tr_x_std = scaler_x.transform(tr_x)
    va_x_std = scaler_x.transform(va_x)
    
    # 学習の実行、バリデーションデータの予測値の出力、スコア計算
    # ランダムフォレスト
    model = RandomForestClassifier(n_estimators=100, max_depth=5, min_samples_split=2, min_samples_leaf=1)
    model.fit(tr_x_std, tr_y)
    # predict_probaで1の確率を配列で渡す
    va_pred = model.predict_proba(va_x_std)[:, 1]
    # roc aucで評価
    score = roc_auc_score(va_y, va_pred)
    scores.append(score)
    
print(np.mean(scores))

0.7213289295710005


In [58]:
# クロスバリデーション4+ランダムフォレストn_estimators=150
kf = KFold(n_splits=4, shuffle=True, random_state=71)
scores = [] 
for tr_idx, va_idx in kf.split(X_train):
    tr_x, va_x = X_train[tr_idx], X_train[va_idx]
    tr_y, va_y = y_train[tr_idx], y_train[va_idx]
    # 標準化
    scaler_x = StandardScaler()
    # Xのみ標準化
    scaler_x.fit(tr_x)
    tr_x_std = scaler_x.transform(tr_x)
    va_x_std = scaler_x.transform(va_x)
    
    # 学習の実行、バリデーションデータの予測値の出力、スコア計算
    # ランダムフォレスト
    model = RandomForestClassifier(n_estimators=150, max_depth=5, min_samples_split=2, min_samples_leaf=1)
    model.fit(tr_x_std, tr_y)
    # predict_probaで1の確率を配列で渡す
    va_pred = model.predict_proba(va_x_std)[:, 1]
    # roc aucで評価
    score = roc_auc_score(va_y, va_pred)
    scores.append(score)
    
print(np.mean(scores))

0.7213946629493284


In [59]:
# クロスバリデーション4+ランダムフォレストn_estimators=300
kf = KFold(n_splits=4, shuffle=True, random_state=71)
scores = [] 
for tr_idx, va_idx in kf.split(X_train):
    tr_x, va_x = X_train[tr_idx], X_train[va_idx]
    tr_y, va_y = y_train[tr_idx], y_train[va_idx]
    # 標準化
    scaler_x = StandardScaler()
    # Xのみ標準化
    scaler_x.fit(tr_x)
    tr_x_std = scaler_x.transform(tr_x)
    va_x_std = scaler_x.transform(va_x)
    
    # 学習の実行、バリデーションデータの予測値の出力、スコア計算
    # ランダムフォレスト
    model = RandomForestClassifier(n_estimators=300, max_depth=5, min_samples_split=2, min_samples_leaf=1)
    model.fit(tr_x_std, tr_y)
    # predict_probaで1の確率を配列で渡す
    va_pred = model.predict_proba(va_x_std)[:, 1]
    # roc aucで評価
    score = roc_auc_score(va_y, va_pred)
    scores.append(score)
    
print(np.mean(scores))

0.7212446744632026


## 【問題5】最終的なモデルの選定
最終的にこれは良いというモデルを選び、推定した結果をKaggleに提出してスコアを確認してください。<br>
どういったアイデアを取り入れ、どの程度のスコアになったかを記載してください。

In [65]:
# クロスバリデーション5+ランダムフォレストn_estimators=150
kf = KFold(n_splits=5, shuffle=True, random_state=71)
model = RandomForestClassifier(n_estimators=150, max_depth=5, min_samples_split=2, min_samples_leaf=1)
# 標準化
scaler_x = StandardScaler()
scores = [] 
for tr_idx, va_idx in kf.split(X_train):
    tr_x, va_x = X_train[tr_idx], X_train[va_idx]
    tr_y, va_y = y_train[tr_idx], y_train[va_idx]
    
    # Xのみ標準化
    scaler_x.fit(tr_x)
    tr_x_std = scaler_x.transform(tr_x)
    va_x_std = scaler_x.transform(va_x)
    
    # 学習の実行、バリデーションデータの予測値の出力、スコア計算
    # ランダムフォレスト
    model.fit(tr_x_std, tr_y)
    # predict_probaで1の確率を配列で渡す
    va_pred = model.predict_proba(va_x_std)[:, 1]
    # roc aucで評価
    score = roc_auc_score(va_y, va_pred)
    scores.append(score)
X_test_std = scaler_x.transform(X_test)
y_pred = model.predict_proba(X_test_std)[:, 1]

In [66]:
print(np.mean(scores))

0.7212282468017754


In [67]:
submission = pd.DataFrame({"SK_ID_CURR": df_copy["SK_ID_CURR"], "TARGET": y_pred})
submission.to_csv("submission_5.csv", index=False)

【回答】ランダムフォレストでパラメータを（n_estimators=150,max_depth=50）で設定、欠損値は平均で置き換え<br>
クロスバリデーションfold4で学習したモデルでkaggleへ提出したところPublic Scoreは0.70768であった<br>
クロスバリデーションfold5に変更したモデルでは0.70889であった<br>
week4の授業課題では調整前で0.50367、調整後に0.68014だったので、0.2ほどスコアをのばせた。<br>
しかし、現状では2年前の順位で6100位ほどで上位には遠く及ばなかった